<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/336_SalesEnab_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Enablement Orchestrator

In [ ]:
"""Sales Enablement Orchestrator

LangGraph workflow that connects all nodes in a linear sequence.
"""

from langgraph.graph import StateGraph, END
from typing import Dict, Any
from config import SalesEnablementOrchestratorState, SalesEnablementOrchestratorConfig
from agents.sales_enablement.nodes import (
    goal_node,
    planning_node,
    data_loading_node,
    lead_prioritization_node,
    customer_needs_analysis_node,
    outreach_generation_node,
    follow_up_coordination_node,
    deal_insights_node,
    rep_nudging_node,
    historical_insights_node,
    executive_reporting_node,
    report_generation_node
)


def create_sales_enablement_orchestrator(
    config: SalesEnablementOrchestratorConfig = None
) -> StateGraph:
    """
    Create the Sales Enablement Orchestrator workflow.

    Args:
        config: Optional configuration (uses default if not provided)

    Returns:
        Compiled LangGraph workflow
    """
    if config is None:
        config = SalesEnablementOrchestratorConfig()

    # Create StateGraph
    workflow = StateGraph(SalesEnablementOrchestratorState)

    # Add all nodes
    workflow.add_node("goal", goal_node)
    workflow.add_node("planning", planning_node)
    workflow.add_node("data_loading", data_loading_node)
    workflow.add_node("lead_prioritization", lead_prioritization_node)
    workflow.add_node("customer_needs_analysis", customer_needs_analysis_node)
    workflow.add_node("outreach_generation", outreach_generation_node)
    workflow.add_node("follow_up_coordination", follow_up_coordination_node)
    workflow.add_node("deal_insights", deal_insights_node)
    workflow.add_node("rep_nudging", rep_nudging_node)
    workflow.add_node("historical_insights", historical_insights_node)
    workflow.add_node("executive_reporting", executive_reporting_node)
    workflow.add_node("report_generation", report_generation_node)

    # Linear workflow: connect all nodes in sequence
    workflow.set_entry_point("goal")
    workflow.add_edge("goal", "planning")
    workflow.add_edge("planning", "data_loading")
    workflow.add_edge("data_loading", "lead_prioritization")
    workflow.add_edge("lead_prioritization", "customer_needs_analysis")
    workflow.add_edge("customer_needs_analysis", "outreach_generation")
    workflow.add_edge("outreach_generation", "follow_up_coordination")
    workflow.add_edge("follow_up_coordination", "deal_insights")
    workflow.add_edge("deal_insights", "rep_nudging")
    workflow.add_edge("rep_nudging", "historical_insights")
    workflow.add_edge("historical_insights", "executive_reporting")
    workflow.add_edge("executive_reporting", "report_generation")
    workflow.add_edge("report_generation", END)

    return workflow.compile()


def run_sales_enablement_orchestrator(
    lead_id: str = None,
    rep_id: str = None,
    focus_area: str = None,
    config: SalesEnablementOrchestratorConfig = None
) -> Dict[str, Any]:
    """
    Run the Sales Enablement Orchestrator with given inputs.

    Args:
        lead_id: Optional specific lead to analyze
        rep_id: Optional specific rep to analyze
        focus_area: Optional focus area ("lead_prioritization", "outreach_generation", etc.)
        config: Optional configuration

    Returns:
        Final state after workflow execution
    """
    # Create orchestrator
    orchestrator = create_sales_enablement_orchestrator(config)

    # Initialize state
    initial_state: SalesEnablementOrchestratorState = {
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_area": focus_area,
        "errors": []
    }

    # Run workflow
    final_state = orchestrator.invoke(initial_state)

    return final_state



# Main entry point for Sales Enablement Orchestrator

In [ ]:
"""Main entry point for Sales Enablement Orchestrator

Run this script to execute the complete sales enablement workflow.
"""

import sys
from datetime import datetime
from agents.sales_enablement.orchestrator import run_sales_enablement_orchestrator
from config import SalesEnablementOrchestratorConfig


def main():
    """Main entry point"""
    print("=" * 60)
    print("Sales Enablement Orchestrator")
    print("=" * 60)
    print()

    # Parse command line arguments (optional)
    lead_id = None
    rep_id = None
    focus_area = None

    if len(sys.argv) > 1:
        # Simple argument parsing
        for arg in sys.argv[1:]:
            if arg.startswith("--lead-id="):
                lead_id = arg.split("=")[1]
            elif arg.startswith("--rep-id="):
                rep_id = arg.split("=")[1]
            elif arg.startswith("--focus-area="):
                focus_area = arg.split("=")[1]

    # Create config (can customize here)
    config = SalesEnablementOrchestratorConfig()

    # Run orchestrator
    print(f"🚀 Starting Sales Enablement Orchestrator...")
    print(f"   Lead ID: {lead_id or 'All leads'}")
    print(f"   Rep ID: {rep_id or 'All reps'}")
    print(f"   Focus Area: {focus_area or 'All areas'}")
    print()

    start_time = datetime.now()

    try:
        final_state = run_sales_enablement_orchestrator(
            lead_id=lead_id,
            rep_id=rep_id,
            focus_area=focus_area,
            config=config
        )

        end_time = datetime.now()
        duration = (end_time - start_time).total_seconds()

        # Print results
        print("=" * 60)
        print("✅ Orchestrator Execution Complete")
        print("=" * 60)
        print()

        # Summary
        if "report_file_path" in final_state:
            print(f"📄 Report saved to: {final_state['report_file_path']}")

        if "pipeline_summary" in final_state:
            summary = final_state["pipeline_summary"]
            print(f"📊 Pipeline: {summary.get('active_deals', 0)} active deals, ${summary.get('total_pipeline_value', 0):,.0f} value")

        if "top_priority_leads" in final_state:
            print(f"🎯 Top Priority Leads: {len(final_state['top_priority_leads'])}")

        if "rep_nudges" in final_state:
            print(f"🔔 Rep Nudges: {len(final_state['rep_nudges'])}")

        if "errors" in final_state and final_state["errors"]:
            print(f"⚠️  Errors: {len(final_state['errors'])}")
            for error in final_state["errors"]:
                print(f"   - {error}")

        print(f"⏱️  Duration: {duration:.2f} seconds")
        print()

        return 0

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return 1


if __name__ == "__main__":
    sys.exit(main())



# Test Suite for Phase 7: Orchestrator Workflow

In [ ]:
"""Test Suite for Phase 7: Orchestrator Workflow"""

from agents.sales_enablement.orchestrator import (
    create_sales_enablement_orchestrator,
    run_sales_enablement_orchestrator
)
from config import SalesEnablementOrchestratorState


def test_create_orchestrator():
    """Test orchestrator creation"""
    print("Testing create_orchestrator...")

    orchestrator = create_sales_enablement_orchestrator()

    assert orchestrator is not None, "should create orchestrator"

    print(f"✅ create_orchestrator test passed!")
    print(f"   Orchestrator created successfully")
    print()


def test_run_orchestrator_full():
    """Test running the full orchestrator workflow"""
    print("Testing run_orchestrator_full...")

    final_state = run_sales_enablement_orchestrator()

    # Check that all expected outputs are present
    assert "goal" in final_state, "should have goal"
    assert "plan" in final_state, "should have plan"
    assert "leads" in final_state, "should have leads"
    assert "prioritized_leads" in final_state, "should have prioritized_leads"
    assert "customer_needs_analysis" in final_state, "should have customer_needs_analysis"
    assert "outreach_recommendations" in final_state, "should have outreach_recommendations"
    assert "follow_up_actions" in final_state, "should have follow_up_actions"
    assert "rep_nudges" in final_state, "should have rep_nudges"
    assert "deal_insights" in final_state, "should have deal_insights"
    assert "win_patterns" in final_state, "should have win_patterns"
    assert "pipeline_summary" in final_state, "should have pipeline_summary"
    assert "enablement_report" in final_state, "should have enablement_report"
    assert "report_file_path" in final_state, "should have report_file_path"

    # Check for errors
    errors = final_state.get("errors", [])
    assert len(errors) == 0, f"should have no errors, but got: {errors}"

    print(f"✅ run_orchestrator_full test passed!")
    print(f"   Report: {final_state.get('report_file_path', 'N/A')}")
    print(f"   Top leads: {len(final_state.get('top_priority_leads', []))}")
    print(f"   Nudges: {len(final_state.get('rep_nudges', []))}")
    print(f"   Pipeline: {final_state.get('pipeline_summary', {}).get('active_deals', 0)} active deals")
    print()


def test_run_orchestrator_with_lead_id():
    """Test running orchestrator with specific lead_id"""
    print("Testing run_orchestrator_with_lead_id...")

    final_state = run_sales_enablement_orchestrator(lead_id="L-001")

    assert final_state.get("lead_id") == "L-001", "should have lead_id in state"
    assert "prioritized_leads" in final_state, "should have prioritized_leads"

    print(f"✅ run_orchestrator_with_lead_id test passed!")
    print(f"   Lead ID: {final_state.get('lead_id')}")
    print()


def test_run_orchestrator_with_focus_area():
    """Test running orchestrator with specific focus_area"""
    print("Testing run_orchestrator_with_focus_area...")

    final_state = run_sales_enablement_orchestrator(focus_area="lead_prioritization")

    assert final_state.get("focus_area") == "lead_prioritization", "should have focus_area in state"
    assert "goal" in final_state, "should have goal"

    print(f"✅ run_orchestrator_with_focus_area test passed!")
    print(f"   Focus Area: {final_state.get('focus_area')}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 7: Orchestrator Workflow - Test Suite")
    print("=" * 60)
    print()

    test_create_orchestrator()
    test_run_orchestrator_full()
    test_run_orchestrator_with_lead_id()
    test_run_orchestrator_with_focus_area()

    print("=" * 60)
    print("✅ All Phase 7 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python run_sales_enablement.py
============================================================
Sales Enablement Orchestrator
============================================================

🚀 Starting Sales Enablement Orchestrator...
   Lead ID: All leads
   Rep ID: All reps
   Focus Area: All areas

============================================================
✅ Orchestrator Execution Complete
============================================================

📄 Report saved to: output/sales_enablement_reports/sales_enablement_20251229_171431_20251229_171431.md
📊 Pipeline: 11 active deals, $0 value
🎯 Top Priority Leads: 10
🔔 Rep Nudges: 24
⏱️  Duration: 0.05 seconds



# # Sales Enablement Report

**Generated:** 2025-12-29 17:15:30  
**Objective:** Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights

---

## Executive Summary

### Pipeline Health

- **Total Deals:** 15
- **Active Deals:** 11
- **Won Deals:** 2
- **Lost Deals:** 2
- **Total Pipeline Value:** \$0
- **Weighted Pipeline Value:** \$0
- **Average Deal Size:** $0
- **Average Days to Close:** 0.0 days
- **Win Rate:** 50.0%
- **Stalled Deals:** 2
- **At-Risk Deals:** 5

### Rep Performance

⚠️ **Alex Morgan** (SR-01)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 81.0%
   - Nudges: 2

⚠️ **Jordan Lee** (SR-02)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 65.3%
   - Nudges: 0

⚠️ **Priya Shah** (SR-03)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 95.0%
   - Nudges: 3

⚠️ **Miguel Alvarez** (SR-04)
   - Active Deals: 0
   - Pipeline Value: $0
   - Close Rate: 0.0%
   - Quota Achievement: 54.0%
   - Nudges: 2

---

## Top Priority Leads

1. **Orion Aerospace** (L-020) - Score: 86.6
2. **Apex Manufacturing** (L-003) - Score: 85.5
3. **Atlas Freight** (L-011) - Score: 83.0
4. **NovaEnergy Solutions** (L-006) - Score: 81.8
5. **OmniPharma** (L-015) - Score: 77.5
6. **Northstar Logistics** (L-001) - Score: 74.5
7. **Vertex Consulting** (L-014) - Score: 71.8
8. **Skyline Retail Group** (L-005) - Score: 70.2
9. **Horizon AgriTech** (L-018) - Score: 69.3
10. **ClearWave Health** (L-002) - Score: 62.6

---

## Customer Needs Analysis

### Lead L-001

- **Pain Points:** manual reporting, forecast inaccuracy
- **Buying Signals:** positive engagement
- **Product Fit Score:** 0.00

### Lead L-002

- **Pain Points:** data silos, slow reporting
- **Product Fit Score:** 0.00

### Lead L-003

- **Pain Points:** cost overruns, margin pressure
- **Buying Signals:** positive engagement, pricing discussed, proposal requested
- **Product Fit Score:** 0.00

### Lead L-004

- **Pain Points:** budget forecasting, grant reporting
- **Product Fit Score:** 0.00

### Lead L-005

- **Pain Points:** inventory forecasting, seasonality
- **Product Fit Score:** 0.00

---

## Outreach Recommendations

- **L-020** → SR-04 via email (N/A)
- **L-003** → SR-01 via email (N/A)
- **L-011** → SR-01 via email (N/A)
- **L-006** → SR-01 via email (N/A)
- **L-015** → SR-03 via call (N/A)

---

## Follow-up Actions

### ⚠️ Overdue (7)

- L-001 → SR-01: schedule_call
- L-003 → SR-01: send_proposal
- L-005 → SR-03: send_pricing
- L-008 → SR-04: schedule_call
- L-009 → SR-04: send_proposal

---

## Rep Nudges

- **Follow Up Due:** 7
- **Stalled Deal:** 2
- **High Priority Lead:** 10
- **Deal At Risk:** 5

### Sample Nudges

- **SR-01:** ⚠️ Follow-up with Northstar Logistics (L-001) is overdue. You promised to schedule demo 35 days ago....
- **SR-01:** ⚠️ Follow-up with Apex Manufacturing (L-003) is overdue. You promised to review proposal internally ...
- **SR-03:** ⚠️ Follow-up with Skyline Retail Group (L-005) is overdue. You promised to send pricing comparison 3...
- **SR-04:** ⚠️ Follow-up with Helix Biotech (L-008) is overdue. You promised to schedule follow-up 40 days ago....
- **SR-04:** ⚠️ Follow-up with Ironclad Construction (L-009) is overdue. You promised to review proposal 43 days ...

---

## Deal Insights

### Stalled Deals (2)

- D-004 (L-005): 21 days in current stage
- D-008 (L-009): 22 days in current stage

### At-Risk Deals (5)

- D-002 (L-002): pricing sensitivity
- D-004 (L-005): discount pressure, negative sentiment in recent interactions
- D-007 (L-008): budget uncertainty
- D-008 (L-009): timeline risk
- D-014 (L-015): legal review

---

## Historical Insights

### Win Patterns

- **Interaction Frequency:** Won deals had 1.0 interactions on average (range: 1-1) (100% frequency)
  → Recommendation: Aim for 1+ interactions for similar deals
- **Positive Sentiment:** 0/2 won deals had 2+ positive interactions (0% frequency)
  → Recommendation: Focus on building positive engagement early in the sales cycle
- **Pricing Discussion:** 0/2 won deals discussed pricing (0% frequency)
  → Recommendation: Introduce pricing discussion early for qualified leads

### Loss Patterns

- **Negative Sentiment:** 1/2 lost deals had negative sentiment (50% frequency)
  → Recommendation: Address negative sentiment immediately when detected
- **Risk Flags:** Most common risk flag in lost deals: 'pricing loss' (1 occurrences) (50% frequency)
  → Recommendation: Monitor and address 'pricing loss' risk flags proactively
- **Competition:** Most common competitor in lost deals: 'VendorX' (1 occurrences) (50% frequency)
  → Recommendation: Develop competitive differentiation strategy against 'VendorX'

---

*Report generated by Sales Enablement Orchestrator Agent*
